<a href="https://colab.research.google.com/github/WB-Jang/Embedding-model-fine-tuning/blob/main/Embedding_model_fine_tuning_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install sentence_transformers
!pip install torch
!pip install peft
!pip install huggingface_hub

In [2]:
from huggingface_hub import snapshot_download

model_id = "BAAI/bge-m3"
snapshot_download(repo_id=model_id, local_dir="./bge-m3", local_dir_use_symlinks=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

'/content/bge-m3'

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import torch
from peft import LoraConfig, get_peft_model

In [4]:
# ---------------------------------------------------------
# 2. 데이터 준비 (Data Preparation)
# ---------------------------------------------------------
data = pd.read_excel("20251205_시사경제용어사전.xlsx")
df = pd.DataFrame(data)
df[['용어','설명']]

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,용어,설명
0,0.5인 가구,싱글족 가운데 두 곳 이상에 거처를 두거나 잦은 여행과 출장 등으로 오랫동안 집을 ...
1,1인 창조기업,"개인이 사장이면서 직원인 기업을 의미한다. 자신이 가진 '지식, 경험, 기술' 등을..."
2,1인당 국민소득,국민소득을 총국민 수로 나눈 값. 해당 국가의 소득 수준을 보여주는 가장 대표적인 ...
3,20-20-20 계획,"유럽연합(EU)이 2020년까지 온실가스 20% 감축, 에너지효율 20% 개선, 신..."
4,2차 시장(Secondary Market),"2차 시장은 처음 발행된 증권, 채권 등이 거래되는 발행시장과 구분되며, 이미 발행..."
...,...,...
3026,흑자도산,일반적으로 기업의 도산은 경영 부진으로 적자가 과도하게 발생할 때 일어난다. 도산의...
3027,희망키움통장,일하는 수급자가구와 비수급 근로빈곤층의 자활을 돕기 위한 자금. 기초생활보장 수급자...
3028,희토류(Rare Earth Metals),"주기율표의 17개 화학 원소의 통칭으로, 스칸듐(Sc)과 이트륨(Y), 그리고 란타..."
3029,히든챔피언,숨은 강소기업. 세계 시장점유율이 1~3위면서 일반에 잘 알려지지 않은 세계적인 경...


In [5]:
df_filtered=df[['용어','설명']]

In [6]:
MODEL_ID = "BAAI/bge-m3"
OUTPUT_PATH = "./finetuned_finance_model_lora"
BATCH_SIZE = 4
NUM_EPOCHS = 3

In [7]:
train_examples = []
for i, row in df_filtered.iterrows():
    term = row['용어']
    definition = row['설명']

    queries = [
        f"{term}",
        f"{term}의 뜻은?",
        f"{term}이란 무엇인가?",
        f"금융 용어 {term} 설명"
    ]

    for query in queries:
        train_examples.append(InputExample(texts=[query, definition]))

In [8]:
# ---------------------------------------------------------
# 3. SentenceTransformer + LoRA 설정
# ---------------------------------------------------------
print(f"모델 로드 중: {MODEL_ID} ...")
model = SentenceTransformer(MODEL_ID)

# SentenceTransformer 내부의 transformer backbone 가져오기
# (BGE 계열은 일반적으로 첫 번째 모듈이 Transformer 입니다.)
base_module = model._first_module()  # sentence_transformers.models.Transformer
hf_model = base_module.auto_model     # transformers 모델 (e.g., BertModel/RoFormer 등)

# LoRA 설정
lora_config = LoraConfig(
    r=8,                # 랭크 (작을수록 가벼움)
    lora_alpha=16,      # LoRA scaling
    target_modules=["query", "value", "key"],  # attention module에만 적용 (모델 구조에 따라 조정 필요)
    lora_dropout=0.05,
    bias="none",
    task_type="FEATURE_EXTRACTION",  # 임베딩/인코더 용도
)

hf_model = get_peft_model(hf_model, lora_config)

# SentenceTransformer 내부 모델에 다시 세팅
base_module.auto_model = hf_model


모델 로드 중: BAAI/bge-m3 ...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [9]:
# 기존 파라미터는 freeze, LoRA 파라미터만 학습
for name, param in hf_model.named_parameters():
    if "lora_" not in name:
        param.requires_grad = False

print("학습 가능한 파라미터 수:")
trainable_params = sum(p.numel() for p in hf_model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in hf_model.parameters())
print(f"  trainable: {trainable_params:,} / total: {all_params:,}")

학습 가능한 파라미터 수:
  trainable: 1,179,648 / total: 568,934,400


In [10]:
# ---------------------------------------------------------
# 4. 데이터 로더 & 손실 함수
# ---------------------------------------------------------
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)
# 배치 사이즈가 너무 작으면 추후에 조정해 볼 것
train_loss = losses.MultipleNegativesRankingLoss(model=model)


In [11]:
# ---------------------------------------------------------
# 5. 학습 시작 (Training)
# ---------------------------------------------------------
print("학습을 시작합니다... (LoRA 기반 경량 파인튜닝)")

warmup_steps = int(len(train_dataloader) * NUM_EPOCHS * 0.1)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=NUM_EPOCHS,
    warmup_steps=warmup_steps,
    output_path=OUTPUT_PATH,
    show_progress_bar=True
)

print(f"\n학습 완료! LoRA 파라미터가 포함된 모델이 '{OUTPUT_PATH}'에 저장되었습니다.")

학습을 시작합니다... (LoRA 기반 경량 파인튜닝)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eeariorie (eeariorie-standard-chartered) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.092700
1000,0.040900
1500,0.029900
2000,0.016300
2500,0.023200
3000,0.016900
3500,0.014600
4000,0.017000
4500,0.013800
5000,0.010800



학습 완료! LoRA 파라미터가 포함된 모델이 './finetuned_finance_model_lora'에 저장되었습니다.


In [19]:
# ---------------------------------------------------------
# 6. 간단 검증 (Validation)
# ---------------------------------------------------------
print("\n[성능 테스트]")
finetuned_model = SentenceTransformer(OUTPUT_PATH)

test_query = "은행의 건전성을 보여주는 지표는?"
docs = [
    "국내총생산은 한 나라의 경제 규모를 나타낸다.",
    "국제결제은행이 정한 위험자산 대비 자기자본비율로 은행 건전성 핵심 지표이다.",  # 정답
    "RP는 환매조건부채권을 의미한다."
]

embeddings_query = finetuned_model.encode(test_query)
embeddings_docs = finetuned_model.encode(docs)

similarities = finetuned_model.similarity(embeddings_query, embeddings_docs)
print(f"질문: {test_query}")
for i, doc in enumerate(docs):
    print(f"문서 {i+1} 유사도: {similarities[0][i]:.4f} -> {doc}")


[성능 테스트]
질문: 은행의 건전성을 보여주는 지표는?
문서 1 유사도: 0.3156 -> 국내총생산은 한 나라의 경제 규모를 나타낸다.
문서 2 유사도: 0.6717 -> 국제결제은행이 정한 위험자산 대비 자기자본비율로 은행 건전성 핵심 지표이다.
문서 3 유사도: 0.0946 -> RP는 환매조건부채권을 의미한다.


In [20]:
# ---------------------------------------------------------
# 7. Ablation Study
# ---------------------------------------------------------
print("\n[성능 테스트]")
original_model = SentenceTransformer(MODEL_ID)

test_query = "은행의 건전성을 보여주는 지표는?"
docs = [
    "국내총생산은 한 나라의 경제 규모를 나타낸다.",
    "국제결제은행이 정한 위험자산 대비 자기자본비율로 은행 건전성 핵심 지표이다.",  # 정답
    "RP는 환매조건부채권을 의미한다."
]

embeddings_query = original_model.encode(test_query)
embeddings_docs = original_model.encode(docs)

similarities = original_model.similarity(embeddings_query, embeddings_docs)
print(f"질문: {test_query}")
for i, doc in enumerate(docs):
    print(f"문서 {i+1} 유사도: {similarities[0][i]:.4f} -> {doc}")


[성능 테스트]
질문: 은행의 건전성을 보여주는 지표는?
문서 1 유사도: 0.4273 -> 국내총생산은 한 나라의 경제 규모를 나타낸다.
문서 2 유사도: 0.7049 -> 국제결제은행이 정한 위험자산 대비 자기자본비율로 은행 건전성 핵심 지표이다.
문서 3 유사도: 0.3925 -> RP는 환매조건부채권을 의미한다.


# Ablation Study 결과
1. 연관성이 높은 항목에 대해 Cosine Similarity가 높아지는 것이 아니라,
2. 연관 없는 것에 대해 Cosine Similarity가 낮아지는 예상치 못한 결과
3. 단, 여러 번의 테스트가 더 필요함